# Nuclear segmentation
In this notebook, we use TGMM (McDole et al. 2018) to segment our data from the SPIM.
To that end, we are using `tgmm_utility.py` to call the segmentation software and to distribute the tasks across the cores of the computer, to accelerate the segementation.

In [1]:
import glob
import os
import datetime
import subprocess
from multiprocessing import Pool
import numpy as np
from pathlib import Path
import glob
import pandas as pd
from scipy.spatial import KDTree
import tqdm

from SegmentationIO import read_tgmm
from tgmm_utility import RunTGMM


## Parameters
First, we define the parameters of the segmentation.

In [2]:
params = {
    'imgFilePattern': None,  # This is the path to the .klb or .tif file of the nucelar channel
    'anisotropyZ':1,  # x/y resolution divided by z resolution (e.g. 0.5um/pixel / 1um/pixel = 2), set to 1 for isotropic resolution
    'backgroundThreshold': None,  # voxels below this threshold are background, measure for every image individually
    'persistanceSegmentationTau':5,  # watershed level, smaller -> more sensitive (oversegmentation), larger -> less sensitive (merged nuclei)
    'maxIterEM':100,  # number of iterations for the GMM fit, reduce to speed up the segmentation
    'tolLikelihood':1e-6,  # stopping criterion for GMM fit, increase to speed up, decrease to increase precision
    'regularizePrecisionMatrixConstants_lambdaMin':0.02,  # minimum eigenvalue of the GMM precision matrix (inverse covariance matrix), smaller values allow for larger nuclei.
    'regularizePrecisionMatrixConstants_lambdaMax':0.1,  # maximum eigenvalue of the GMM precision matrix (inverse covariance matrix), larger values allow for smaller nuclei.
    'regularizePrecisionMatrixConstants_maxExcentricity':9.0,  # maximum Excentricity of the fittet GMM (longest axis / shortest axis)
    'radiusMedianFilter':1,  # radius of a median filter applied pre-watershed for de-noising, increase if image is noisy, however, small objects will be lost
    'minTau':2,  # minimum watershed level
    'conn3D':74,  # voxel neighbourhood considered for watershed segmentation, allowed are 6, 28 and 74
    'minNucleiSize':8,  # minimum volume (in voxels) of a single nucleus, smaller objects will be disregarded
    'maxNucleiSize':3000,  # maximum volume (in voxels) of a single nucleus, larger objects will be disregarded
    'maxPercentileTrimSV':0.2,  # maximum percentage of voxels in a watershedded region that are identified as nucleus, decrease for sparse regions, increase for dense regeions
    'conn3DsvTrim':6,  # I honestly don't know, but also this doesn't seem to change the result
    
    'betaPercentageOfN_k':0.05,  # for tracking, unused
    'nuPercentageOfN_k':1.0,  # for tracking, unused
    'alphaPercentage':0.7,  # for tracking, unused
    'temporalWindowForLogicalRules':5,  # for tracking, unused
    'thrBackgroundDetectorHigh':1.1,  # for tracking, unused
    'thrBackgroundDetectorLow':0.2,  # for tracking, unused
    'SLD_lengthTMthr':5,  # for tracking, unused
    'estimateOpticalFlow':0,  # for tracking, unused
    'maxDistPartitionNeigh':80.0,  # for tracking, unused
    'deathThrOpticalFlow':-1,  # for tracking, unused
    'maxNumKNNsupervoxel':10,  # for tracking, unused
    'maxDistKNNsupervoxel':41.0,  # for tracking, unused
    'thrSplitScore':-1.0,  # for tracking, unused
    'thrCellDivisionPlaneDistance':12.403,  # for tracking, unused
    'thrCellDivisionWithTemporalWindow':0.456,  # for tracking, unused
}


We store the paramters in a csv file, to have acces to them later.

In [8]:
dd = pd.DataFrame.from_dict(params, orient='index')
dd.to_csv('params.csv')

## Organisation of image files and paths
Next, we set up the required paths to run the segmentations. 

In [ ]:
# path to the folder conatining the nuclear images to be segmented (as .klb or .tif)
klb_path = Path('path/to/imgages')
files = list(klb_path.glob('*.tif'))  # replace '*.tif' with '*.klb', depending on the file type.

# path to folder where the segmentation outputs will be saved, needs to exist
output_directory = Path('path/to/segmentation/output')

# path to TGMM executables (bin folder in the TGMM software, containing the .exe files)
local_executables = Path(r'path/to/TGMM/Tracking_GMM_project-v0.3.0-win64/bin')


## Background trhesholds
The TGMM software uses thresholding to filter out background noise. A threshold has to be provided for each image individually. We recommend using FIJI for this purpose. Our workflow was:
* Generate the csv file 'bg_thresholds.csv' by running the cell below. This will generate a table with the image paths and the measured threshold (initialised to 0).
* Open this table in an external text editor.
* Open the nucelar images to be segmented in FIJI one by one.
* Measure the background levles in a non-fluorescend part of the sample (e.g. the yolk)
* Used the 'adjust contrast' tool to test the identified threshold.
* Update the threshold in the 'bg_thresholds.csv' file.
* Repeat for all images.
* Save csv file as 'bg_thresholds_filled_in.csv'.
* Continue running the notebook.

It is important to set this threshold correctly. Too high values will lead to drastic undersegmentation in dim regions and too low values will lead to false positives in noisy regions. The segmentation will also take significantly longer/shorter for too low/high values.

In [5]:
bg_threshold_file = pd.DataFrame(files, columns=['file'])
bg_threshold_file['threshold'] = 0
bg_threshold_file.to_csv(str(klb_path/'bg_thresholds.csv'))

In [ ]:
bg_threshold = pd.read_csv(klb_path/'bg_thresholds_filled_in.csv')
threshold_dict = dict(zip(bg_threshold['file'].values, bg_threshold['threshold'].values))

## Setting up the pipeline

Next, we set up the pipeline for the nuclear segmentation. We give two examples in the following. The first one, `pipeline`, simply runs runs TGMM on individual images, while the second one, `pipleine_multiple_runs`, runs the segmentation multiple times for different `maxPercentileTrimSV` values to properly segment regions of different density. The results of the multiple runs are then merged into one output file. Note that the points of the different runs can be differentiated by the `run` column. Results are stored as csv files in the output directory.

In [ ]:
def pipline(file):
    segmentation = RunTGMM(  # generate the object that will manage the segmentation
        name=str(file.stem),  # name of the image
        output_directory=str(output_directory),  # results folder
        local_executables=str(local_executables),  # TGMM executables (bin folder in software package)
        parameters=params,  # previously specified segmentation parameters
    )
    segmentation.update_parameters(  # insert image specific paramters
        {'imgFilePattern':str(file)[:-4],  # path to the image
        'backgroundThreshold':threshold_dict[str(file)]}  # background level of the image
    )
    segmentation.generate_tgmm_config()  # generate a configuration file to run TGMM
    segmentation.run_watershed_segmentation()  # first step: generate watershed segmentation
    segmentation.run_tgmm()  # second step: fit GMMs
    segmentation.get_segmentation_as_df()  # read in the TGMM output as pandas DataFrame
    df.to_csv(str(output_directory / file.stem) + '.csv')  # store as .csv file
    return df

In [2]:
def pipline_multiple_runs(file):
    segmentation = RunTGMM(
        name=str(file.stem),
        output_directory=str(output_directory),
        local_executables=str(local_executables),
        parameters=params,
    )
    segmentation.update_parameters(
        {'imgFilePattern':str(file)[:-4],
        'backgroundThreshold':threshold_dict[str(file)]}
    )
    segmentation.generate_tgmm_config()
    segmentation.run_watershed_segmentation()
    for mptsv in [0.2, 0.5, 0.7]:  # loop over different maxPercentileTrimSV values
        segmentation.update_parameters({'maxPercentileTrimSV': mptsv})  # update the config file
        segmentation.generate_tgmm_config()  # write updated config file
        segmentation.run_tgmm()  # run GMM segmentaiton
    segmentation.get_segmentation_as_df()
    df = segmentation.combine_segmentations()  # combine the runs of different segmentations using a distance metric, points under distance=5 (um) are deleted
    df.to_csv(str(output_directory / file.stem) + '.csv')
    return df

## Run the segmentation on multiple files
Finally, we iterate over the files to generate the segmentations.

In [8]:
for f in tqdm.tqdm(files):
    pipline(f)

100%|████████████████████████████████████████████████████████████████████████████████| 39/39 [1:04:51<00:00, 99.78s/it]
